This is the jupyter notebook for DM&ML Assignment 1 Part B

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score,auc,accuracy_score,classification_report,confusion_matrix

In [2]:
raw_data = pd.read_excel('Forest.xlsx')
print(raw_data.head())

  class  b1  b2  b3   b4  b5   b6   b7  b8  b9  ...  pred_minus_obs_H_b9  \
0    d   39  36  57   91  59  101   93  27  60  ...                -2.36   
1    h   84  30  57  112  51   98   92  26  62  ...                -2.26   
2    s   53  25  49   99  51   93   84  26  58  ...                -1.46   
3    s   59  26  49  103  47   92   82  25  56  ...                 2.68   
4    d   57  49  66  103  64  106  114  28  59  ...                -2.94   

   pred_minus_obs_S_b1  pred_minus_obs_S_b2  pred_minus_obs_S_b3  \
0               -18.41                -1.88                -6.43   
1               -16.27                -1.95                -6.25   
2               -15.92                -1.79                -4.64   
3               -13.77                -2.53                -6.34   
4               -21.74                -1.64                -4.62   

   pred_minus_obs_S_b4  pred_minus_obs_S_b5  pred_minus_obs_S_b6  \
0               -21.03                -1.60                -6.18  

In [3]:
attributes = raw_data.iloc[:,1:].values
labels = raw_data.iloc[:,0].values
pred_train, pred_test, tar_train, tar_test  = train_test_split(attributes,labels,test_size=0.3,random_state=30,stratify=labels)

In [4]:
classifier_DT = DecisionTreeClassifier(criterion="entropy", random_state=999, max_depth=4)
classifier_DT.fit(pred_train,tar_train)
predictions_DT = classifier_DT.predict(pred_test)

In [10]:
print(accuracy_score(tar_test, predictions_DT))
precision = precision_score(y_true=tar_test, y_pred=predictions_DT, average='micro')
print(precision)
recall = recall_score(y_true=tar_test, y_pred=predictions_DT, average='micro')
print(recall)

0.8789808917197452
0.8789808917197452
0.8789808917197452


In [11]:
print(classification_report(tar_test,predictions_DT))

              precision    recall  f1-score   support

          d        0.85      0.92      0.88        48
          h        0.88      0.85      0.86        26
          o        1.00      0.72      0.84        25
          s        0.87      0.93      0.90        58

    accuracy                           0.88       157
   macro avg       0.90      0.85      0.87       157
weighted avg       0.89      0.88      0.88       157



In [12]:
print(confusion_matrix(tar_test,predictions_DT))

[[44  0  0  4]
 [ 0 22  0  4]
 [ 7  0 18  0]
 [ 1  3  0 54]]


In [13]:
MLP_Classifier = MLPClassifier(max_iter=500)
MLP_Classifier.fit(pred_train,np.ravel(tar_train,order='C'))
predictions_MLP = MLP_Classifier.predict(pred_test)
print(predictions_MLP)

['s ' 's ' 's ' 'd ' 's ' 'd ' 's ' 's ' 's ' 'o ' 'd ' 'o ' 'd ' 's '
 's ' 'd ' 'd ' 's ' 'o ' 's ' 's ' 'd ' 'd ' 's ' 'o ' 'd ' 's ' 's '
 'h ' 'o ' 'd ' 'd ' 's ' 's ' 'd ' 's ' 'd ' 'd ' 's ' 's ' 's ' 'd '
 's ' 'h ' 'o ' 'h ' 'h ' 'h ' 'h ' 's ' 'd ' 'd ' 'o ' 'd ' 'd ' 's '
 's ' 's ' 's ' 'o ' 'o ' 'd ' 'h ' 's ' 'd ' 'd ' 'd ' 's ' 'd ' 'd '
 'd ' 's ' 'd ' 'd ' 's ' 'd ' 'd ' 'd ' 'h ' 'h ' 's ' 'd ' 's ' 's '
 'h ' 's ' 'h ' 's ' 'h ' 'd ' 's ' 's ' 's ' 'd ' 'd ' 'd ' 'd ' 's '
 'o ' 'h ' 'd ' 's ' 'd ' 'd ' 'h ' 'd ' 's ' 'o ' 'd ' 'h ' 'd ' 'o '
 'h ' 'o ' 's ' 's ' 's ' 'h ' 'd ' 's ' 'd ' 'o ' 's ' 'h ' 'o ' 'd '
 's ' 's ' 'o ' 's ' 's ' 's ' 'o ' 's ' 's ' 's ' 'd ' 's ' 's ' 's '
 'h ' 's ' 's ' 'd ' 's ' 'o ' 'h ' 'd ' 's ' 'o ' 'o ' 'd ' 's ' 's '
 'h ' 'h ' 's ']


/Users/vatsal/opt/anaconda3/envs/DMML/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
print(classification_report(tar_test,predictions_MLP))

              precision    recall  f1-score   support

          d        0.90      0.94      0.92        48
          h        0.91      0.77      0.83        26
          o        1.00      0.80      0.89        25
          s        0.85      0.95      0.89        58

    accuracy                           0.89       157
   macro avg       0.91      0.86      0.88       157
weighted avg       0.90      0.89      0.89       157



In [15]:
probDT = classifier_DT.predict_proba(pred_test)
probMLP = MLP_Classifier.predict_proba(pred_test)

In [17]:
probabilities = []
for i in range(len(pred_test)):
    probabilities.append({'d':[probDT[i][0],probMLP[i][0]],'h':[probDT[i][1],probMLP[i][1]],'o':[probDT[i][2],probMLP[i][2]],'s':[probDT[i][3],probMLP[i][3]]})
firstSampleProbability = probabilities[0]
print("\t DecisionTree \t\t MLP")
for sampleProbability in probabilities:
    for probs in sampleProbability:
        print(probs,'\t',sampleProbability[probs])
    break


	 DecisionTree 		 MLP
d 	 [0.04032258064516129, 0.006646672282502062]
h 	 [0.008064516129032258, 0.004354964147518743]
o 	 [0.008064516129032258, 0.0003800836479371816]
s 	 [0.9435483870967742, 0.988618279922042]


In [18]:
def averageProbablePrediction(testSet):
    global np
    testSet = np.atleast_2d(testSet)
    averageProbabilities = []
    a,b = classifier_DT.predict_proba(testSet),MLP_Classifier.predict_proba(testSet)
    averages = (np.add(a,b)/2).tolist()
    for avg in averages:
        maxValueIndex = avg.index(max(avg))
        averageProbabilities.append(classifier_DT.classes_[maxValueIndex])
    return np.array(averageProbabilities)
print(averageProbablePrediction(pred_test[:4]))
print(predictions_MLP[:4])

['s ' 's ' 's ' 'd ']
['s ' 's ' 's ' 'd ']
